# Rule Based Association Pattern mining to find the sumarized the terms and condition
Team members:
Nikhil Gola(MT18129) Ridha Juneja(MT18009) Saru Brar(MT18014) Yogesh Pandey(MT18140)
*****************************************************************************************

Importing all the necessary libraries required for the Rule based Association Pattern mining

In [205]:
import numpy as np  
import matplotlib.pyplot as plt  
import pickle
from nltk.stem import WordNetLemmatizer

In [206]:
import nltk
from collections import Counter
import numpy as np

In [207]:
import os
import re
import copy
import time

Setting the min_support and min_confidence value for the Confidence table

In [208]:
min_support=0.0
min_confidence=0.99


Loading the pickle word dictionary pickle file after preprocessing of the data (pickle file will act as a cache to our System)

In [209]:
def loadWordDictionaryPickle():
    f=open('Pickled Data/word_Dictionary.pickle','rb')
    vocab=pickle.load(f)
    f.close()
    return vocab



Loading the word frequncy pickle file from preprocessed data

In [210]:
def loadWordFrequencyPickle():
    f1=open('Pickled Data/word_Frequency.pickle','rb')
    DocVocab=pickle.load(f1)
    #print(DocVocab['Facebook'])
    f1.close()
    return DocVocab


Loading the unique word pickle from the preprocessed data

In [211]:
def loadUniqueWordPickle(): 
    f1=open('Pickled Data/unique_Wordset.pickle','rb')
    unique_word=pickle.load(f1)
    unique_word=list(unique_word)
    f1.close()
    return unique_word


Calculating word frequency in the training set(corpus) and calculating the probability of word for occuring in corpus and the tfidf values for every word in the corpus

In [212]:
def uniqueWordFrequency(DocVocab):
    Word_Set_frequency={}
    wordSetSupport = {}
    tfidf_dict={}
    maxDoc=len(DocVocab.keys())
    print(maxDoc)
    for key in DocVocab.keys():
        for word in DocVocab[key]:
            if Word_Set_frequency.get(word) ==None:
                Word_Set_frequency[word]=DocVocab[key][word]
                if DocVocab[key][word]==0:
                    wordSetSupport[word]=0.0
                else:
                    wordSetSupport[word]=1.0
            else:
                if DocVocab[key][word]==0:
                    wordSetSupport[word]+=0.0
                else:
                    wordSetSupport[word]+=1.0
                Word_Set_frequency[word]+=DocVocab[key][word]
    #print(wordSetSupport['combine'])
    total1_sum=np.sum([Word_Set_frequency[key] for key in Word_Set_frequency.keys()])
    for key in wordSetSupport.keys():
            wordSetSupport[key]=wordSetSupport[key]/maxDoc
            tfidf_dict[key]=wordSetSupport[key]*(Word_Set_frequency[key]/(1.0*total1_sum))
    return Word_Set_frequency,wordSetSupport,tfidf_dict
        

Filtering the words from the wordset based upon their frequency which above the mean of frequency

In [213]:


#print([Word_Set_frequency[key]  for key in Word_Set_frequency.keys()])

#print(mean1)
#median1=np.median([Word_Set_frequency[key]  for key in Word_Set_frequency.keys()])
#print(median1)
#threshold=np.array([Word_Set_frequency[key]  for key in Word_Set_frequency.keys()])
#threshold1=np.reshape(len(threshold),1)
#kmeans = KMeans(n_clusters=4, random_state=0).fit(threshold1)
#print(kmeans)
def filterWordSetFrequencySupport(mean1,Word_Set_frequency,wordSetSupport):
    req_Dic={}
    req_Dic_Sup={}
    for key in Word_Set_frequency.keys():
        if Word_Set_frequency[key]>mean1:
            req_Dic[key]=Word_Set_frequency[key]
    for key in req_Dic.keys():
        req_Dic_Sup[key]=wordSetSupport[key]
    return req_Dic,req_Dic_Sup



In [214]:
'''fp = open('TFIDF_values.pickle','rb')
tfidf_values={}
tfidf = pickle.load(fp)
#print(tfidf['Google'])
for key in tfidf.keys():
    for wordkey in tfidf[key].keys():
        if tfidf_values.get(wordkey)==None:
            tfidf_values[wordkey]=tfidf[key][wordkey]
        else:
            tfidf_values[wordkey]+=tfidf[key][wordkey]
for key in tfidf_values.keys():
    tfidf_values[key] = tfidf_values[key]/len(tfidf.keys())
#print(tfidf_values[''])
'''

"fp = open('TFIDF_values.pickle','rb')\ntfidf_values={}\ntfidf = pickle.load(fp)\n#print(tfidf['Google'])\nfor key in tfidf.keys():\n    for wordkey in tfidf[key].keys():\n        if tfidf_values.get(wordkey)==None:\n            tfidf_values[wordkey]=tfidf[key][wordkey]\n        else:\n            tfidf_values[wordkey]+=tfidf[key][wordkey]\nfor key in tfidf_values.keys():\n    tfidf_values[key] = tfidf_values[key]/len(tfidf.keys())\n#print(tfidf_values[''])\n"

In [215]:
'''def returnItemsWithMinSupport(vocab):
        maxDoc=len(vocab.keys())
        Support_Score={}
        for key in vocab.keys():
            for words in vocab[key]:
                if Support_Score.get(words) is None:
                    Support_Score[words]=1
                else:
                    Support_Score[words]+=1
        for key in Support_Score.keys():
            Support_Score[key]=Support_Score[key]/maxDoc
        return Support_Score
'''

'def returnItemsWithMinSupport(vocab):\n        maxDoc=len(vocab.keys())\n        Support_Score={}\n        for key in vocab.keys():\n            for words in vocab[key]:\n                if Support_Score.get(words) is None:\n                    Support_Score[words]=1\n                else:\n                    Support_Score[words]+=1\n        for key in Support_Score.keys():\n            Support_Score[key]=Support_Score[key]/maxDoc\n        return Support_Score\n'

In [216]:
def getData():
    words=[]
    with open("data1.txt","r") as f:
        x  = f.readlines()
        for word in x:
            words.append(word.replace("\n",""))
            
    return words


In [217]:
def FilterDictionary(A,Support_Score,unique_word):
    filteredDict=[]
    SScore={}
    for word in A:
        if word in unique_word:
            filteredDict.append(word)
            SScore[word]=Support_Score[word]
    return filteredDict,SScore

In [218]:
def LemmatizeWord(A):
    WN=WordNetLemmatizer()
    i=0;
    maxLen=len(A)
    while i<maxLen:
        A[i]=WN.lemmatize(A[i])
        i+=1
    return A
        

In [219]:
def ListToDic(vocab):
    my_dict={}
    for doc in vocab.keys():
        my_dict = {k: 0 for k in vocab[doc] }
        vocab[doc]=my_dict
    
    return vocab

In [220]:
def confidence(A,B,vocab):
    ScoreMatrix=np.zeros((len(A),len(B)))
    #print(len(B))
    #print(len(A))
    for index1,word1 in enumerate(A):# A is list of words
        for index2,word2 in enumerate(B):
            common=0
            totA=0
            for doc in vocab.keys():
                if  vocab[doc].get(word1) is not None :
                    totA+=1
                    if vocab[doc].get(word2) is not None:
                        common+=1
                    
            if totA==0:
                print(word1,word2)
            ScoreMatrix[index1][index2]=common/totA
    return ScoreMatrix

In [221]:

#print(len(A))
#print(requiredDicti['software'])
#print('software' in B )
#print(len(B))
#print(B)
#print(Ascore)

In [230]:
def SelectAssociationRules(confidenceMatrix,min_support,min_confidence,A,B,Support_Score):
    row,col=confidenceMatrix.shape
    #print(row,col)
    required={}
    for i in range(0,row):
        for j in range(0,col):
            word=A[i]
            score=Support_Score[word]
            word2=B[j]
            if confidenceMatrix[i][j]>=min_confidence and score>min_support:
                if required.get(word) ==None and (word!='' and word2!='') :
                    required[word]=[word2]
                else:
                    if (word!='' and word2!=''):
                        required[word].append(word2)
                print(word,'-->',word2)
                #print("\n")
    return required

In [223]:
'''def MeanOFMean(confidenceMatrix):
    mean1=[]
    row,col=confidenceMatrix.shape
    for i in range(0,row):
            mean1.append(np.median(confidenceMatrix[i]))
#             print(mean1)
    return np.mean(mean1)
MeanOFMean(confidenceMatrix)
'''

'def MeanOFMean(confidenceMatrix):\n    mean1=[]\n    row,col=confidenceMatrix.shape\n    for i in range(0,row):\n            mean1.append(np.median(confidenceMatrix[i]))\n#             print(mean1)\n    return np.mean(mean1)\nMeanOFMean(confidenceMatrix)\n'

In [224]:
def Find(string): 
    # findall() has been used  
    # with valid conditions for urls in string 
    url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+] |[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', string) 
    return url 

In [225]:
def printSummary(rule_Dict,outputfile):
    #list_of_lines_to_print = []
    #print(rule_Dict.keys())
    
    #print(rule_Dict['personal_information']['DuckDuckGo does not collect or share personal information'])
    
    #rule_Dict[key][key2] for key2 in rule_Dict[key]
    with open(outputfile,"w") as file1:
        for key in rule_Dict.keys():
            testList=rule_Dict[key]
            #print([ele[0] for ele in testList])
            mean8 = np.mean([ele[0][1] for ele in testList])
            #print(mean8)
            for rows in testList:
                #print(rows[0])
                if rows[0][1]>=mean8:
                    #rows[0][0]=rows[0][0].decode('utf-8','ignore')
                    file1.write(rows[0][0]+'\n')
               
                    #list_of_lines_to_print.append(key2)
                    
            

In [226]:

def doSummarization(filename,filepath,rules,tfidf_dict):
    outputfile = filepath+'summary/'+filename[:-4]+'Summary'+'.txt'
    filename = filepath+filename
    linesData =[]
    rule_Dict={}
    with open(filename,encoding="utf8") as file:
        data = file.readlines()
        #print(data)
        
        for line2 in data:
            linedata=line2.split(".")
            for line in linedata:
                s = Find(line)
                #print("url:", s)
                if len(s)<=0:
                    
                    line = line.replace("\n","")
                    line = line.replace("-","")
                    line1=line.lower()
                    #print("line:",line)
                    tokens=nltk.word_tokenize(line1)
                    if len(tokens)>1:
                        tokens=nltk.word_tokenize(line)
                        #print(tokens)
                        flag=0
                        weight=0
                        line_dict=[]
                        for token in tokens:
                            if rules.get(token)!=None:
                                for word2 in rules[token]:
                                    if word2 in tokens:
                                        flag=1
                                        #print("Printing this line beacuse of ",token,"-->",word2)
                                        #print('--> ',line,".\n")
                                        linesData.append(line)
                                        for token1 in tokens:
                                            if tfidf_dict.get(token1) is not None:
                                                weight=weight+tfidf_dict[token1]
                                        line_dict.append([line,weight])
                                        break
                            if flag==1:
                                break
                        
                        if flag==1:
                                w1=token.lower()
                                w2=word2.lower()
                                str1=w1+'_'+w2
                                if rule_Dict.get(str1) is None:
                                    rule_Dict[str1]=[line_dict]
                                else:
                                    rule_Dict[str1].append(line_dict)

    #print(len(linesData))
   
    printSummary(rule_Dict,outputfile)



In [232]:
def main():
    vocab = loadWordDictionaryPickle()
    Support_Score={}
    DocVocab = loadWordFrequencyPickle()
    Word_Set_frequency,wordSetSupport,tfidf_dict=uniqueWordFrequency(DocVocab)
    mean1=np.mean([Word_Set_frequency[key]  for key in Word_Set_frequency.keys()])
    req_Dic,req_Dic_Sup = filterWordSetFrequencySupport(mean1,Word_Set_frequency,wordSetSupport)
    unique_word = loadUniqueWordPickle()
    vocab=ListToDic(vocab)
    A=getData()
    requiredDicti ={}
    A,Ascore=FilterDictionary(A,wordSetSupport,unique_word)
    A=LemmatizeWord(A)
    for key in req_Dic.keys():
        if key not in A:
            requiredDicti[key]=req_Dic[key]
    B=[]
    for item,value in requiredDicti.items():
        B.append(item)
    confidenceMatrix=confidence(A,B,vocab)
    print(confidenceMatrix)
    rules=SelectAssociationRules(confidenceMatrix,min_support,min_confidence,A,B,wordSetSupport)
    test_path = "Datasets/testdata/"
    test_files = os.listdir(test_path)  
    print(test_files)
    #doSummarization("Datasets/testdata/Github.txt")
    for file in test_files:
        if os.path.isdir(test_path+file):
            pass
        else:
            print("doing for ",file)
            doSummarization(file,test_path,rules,tfidf_dict)


In [233]:
def similarity(file1,file2,name1,name2):
    count=0
    data = open(file2,'rb')
    listoffile=[]
    for line in data:
        listoffile.append(line.lower())
    #print("list:",listoffile)
    data1 = open(file1,'rb')
    for line1 in data1:
        #print("line1:",line1.lower())
        if line1.lower() in listoffile:
            count+=1
    acc = count/len(listoffile)*100
    print("Accuracy for ",name1," and ",name2," is : ",acc,"%")

In [234]:
main()

sum_path = "Datasets/testdata/summary/"
grnd_path ="Datasets/GroundTruth/"
sum_files = os.listdir(sum_path)
grnd_files = os.listdir(grnd_path)

#print(sum_files)
#print(grnd_path)
#doSummarization("Datasets/testdata/Github.txt")
for i in range(0,len(sum_files)):
    if os.path.isdir(sum_path+sum_files[i]):
        pass
    else:
        similarity(sum_path+sum_files[i],grnd_path+grnd_files[i],sum_files[i],grnd_files[i])
        


158
[[1.         0.33333333 0.33333333 ... 0.5        0.         0.5       ]
 [0.25       0.25       0.         ... 0.5        0.         0.25      ]
 [0.42105263 0.15789474 0.28947368 ... 0.28947368 0.02631579 0.26315789]
 ...
 [1.         0.875      0.375      ... 0.875      0.         0.25      ]
 [0.375      0.5        0.375      ... 0.25       0.         0.        ]
 [0.65277778 0.39583333 0.28472222 ... 0.36805556 0.00694444 0.24305556]]
discrimination --> violate
discrimination --> result
discrimination --> user
discrimination --> connection
discrimination --> violation
discrimination --> activity
discrimination --> sexual
discrimination --> include
discrimination --> access
discrimination --> consent
discrimination --> subject
discrimination --> person
discrimination --> purpose
discrimination --> law
discrimination --> additional
discrimination --> available
discrimination --> follow
discrimination --> form
discrimination --> basis
discrimination --> may
discrimination --> pro

separation --> forum
encryption --> cannot
encryption --> user
encryption --> contact
encryption --> also
encryption --> include
encryption --> need
encryption --> access
encryption --> require
encryption --> otherwise
encryption --> purpose
encryption --> law
encryption --> available
encryption --> may
encryption --> protect
encryption --> provide
encryption --> public
encryption --> term
encryption --> obtain
encryption --> party
encryption --> data
encryption --> information
encryption --> make
encryption --> please
encryption --> relate
encryption --> responsible
encryption --> create
encryption --> post
encryption --> use
encryption --> content
encryption --> know
encryption --> link
encryption --> service
encryption --> allow
hazard --> violate
hazard --> bring
hazard --> scope
hazard --> read
hazard --> policy
hazard --> cannot
hazard --> perform
hazard --> vary
hazard --> exercise
hazard --> product
hazard --> exceed
hazard --> proprietary
hazard --> list
hazard --> result
haza

guard --> read
guard --> policy
guard --> manner
guard --> user
guard --> agree
guard --> request
guard --> applicable
guard --> unless
guard --> sale
guard --> portion
guard --> website
guard --> establish
guard --> activity
guard --> right
guard --> measure
guard --> deem
guard --> also
guard --> limit
guard --> accept
guard --> understand
guard --> include
guard --> base
guard --> material
guard --> transaction
guard --> appropriate
guard --> whether
guard --> seek
guard --> notify
guard --> legal
guard --> circumstance
guard --> access
guard --> otherwise
guard --> permit
guard --> partner
guard --> ability
guard --> communication
guard --> consent
guard --> state
guard --> collect
guard --> full
guard --> transmit
guard --> share
guard --> locate
guard --> reasonable
guard --> person
guard --> continue
guard --> purpose
guard --> law
guard --> particular
guard --> compliance
guard --> additional
guard --> either
guard --> electronic
guard --> govern
guard --> via
guard --> availab

spyware --> interact
spyware --> act
spyware --> assume
spyware --> download
spyware --> subscription
spyware --> express
spyware --> hold
spyware --> employee
spyware --> consequential
spyware --> harmless
spyware --> immediate
spyware --> complete
spyware --> failure
spyware --> advertising
spyware --> subsequent
spyware --> submission
spyware --> pertain
spyware --> cover
spyware --> resident
spyware --> indemnify
spyware --> set
spyware --> faith
spyware --> someone
spyware --> receipt
spyware --> analyze
spyware --> specific
spyware --> government
spyware --> form
spyware --> initiate
spyware --> web
spyware --> resource
spyware --> basis
spyware --> platform
spyware --> applies
spyware --> belief
spyware --> market
spyware --> address
spyware --> power
spyware --> derivative
spyware --> may
spyware --> fail
spyware --> area
spyware --> administer
spyware --> export
spyware --> harmful
spyware --> show
spyware --> constitute
spyware --> referral
spyware --> nothing
spyware --> pro

distress --> whatsoever
distress --> share
distress --> locate
distress --> help
distress --> digital
distress --> reasonable
distress --> person
distress --> continue
distress --> purpose
distress --> law
distress --> item
distress --> particular
distress --> pay
distress --> bind
distress --> interfere
distress --> kind
distress --> prohibit
distress --> discretion
distress --> thirty
distress --> exclusive
distress --> disclose
distress --> essential
distress --> compliance
distress --> additional
distress --> idea
distress --> inform
distress --> either
distress --> guarantee
distress --> electronic
distress --> govern
distress --> obligation
distress --> marketing
distress --> guardian
distress --> affect
distress --> via
distress --> available
distress --> herein
distress --> tax
distress --> follow
distress --> remedy
distress --> interact
distress --> act
distress --> assume
distress --> express
distress --> hold
distress --> employee
distress --> consequential
distress --> har

...

discriminate --> give
discriminate --> make
discriminate --> without
discriminate --> please
discriminate --> option
discriminate --> write
discriminate --> change
discriminate --> notice
discriminate --> infringe
discriminate --> free
discriminate --> identify
discriminate --> accordance
discriminate --> us
discriminate --> important
discriminate --> ensure
discriminate --> necessary
discriminate --> responsible
discriminate --> issue
discriminate --> responsibility
discriminate --> telephone
discriminate --> believe
discriminate --> send
discriminate --> comply
discriminate --> time
discriminate --> age
discriminate --> post
discriminate --> reasonab
discriminate --> delete
discriminate --> case
discriminate --> apply
discriminate --> use
discriminate --> content
discriminate --> communicate
discriminate --> review
discriminate --> program
discriminate --> error
discriminate --> choose
discriminate --> control
discriminate --> link
discriminate --> explicit
discriminate --> service
d

affirmative --> relevant
affirmative --> independent
affirmative --> also
affirmative --> delivery
affirmative --> dispute
affirmative --> related
affirmative --> limit
affirmative --> acknowledge
affirmative --> include
affirmative --> base
affirmative --> except
affirmative --> transaction
affirmative --> confirm
affirmative --> appropriate
affirmative --> whether
affirmative --> minor
affirmative --> contain
affirmative --> notify
affirmative --> legal
affirmative --> determine
affirmative --> company
affirmative --> represent
affirmative --> access
affirmative --> direct
affirmative --> require
affirmative --> release
affirmative --> copyright
affirmative --> otherwise
affirmative --> amend
affirmative --> permit
affirmative --> card
affirmative --> communication
affirmative --> consent
affirmative --> provider
affirmative --> international
affirmative --> state
affirmative --> keep
affirmative --> subject
affirmative --> unable
affirmative --> effective
affirmative --> share
affir

trouble --> proprietary
trouble --> list
trouble --> see
trouble --> defend
trouble --> result
trouble --> implied
trouble --> arise
trouble --> experience
trouble --> another
trouble --> sensitive
trouble --> order
trouble --> manner
trouble --> user
trouble --> connection
trouble --> defense
trouble --> agree
trouble --> analytics
trouble --> request
trouble --> long
trouble --> across
trouble --> comment
trouble --> claim
trouble --> must
trouble --> applicable
trouble --> remain
trouble --> possibility
trouble --> search
trouble --> violation
trouble --> updated
trouble --> others
trouble --> example
trouble --> waiver
trouble --> total
trouble --> block
trouble --> unless
trouble --> good
trouble --> charge
trouble --> transfer
trouble --> disclosure
trouble --> economic
trouble --> instruction
trouble --> response
trouble --> title
trouble --> article
trouble --> commit
trouble --> work
trouble --> design
trouble --> nature
trouble --> copy
trouble --> regulatory
trouble --> sale

trouble --> service
trouble --> permission
trouble --> court
trouble --> social
trouble --> source
trouble --> resolution
trouble --> authority
trouble --> along
trouble --> trademark
trouble --> interaction
trouble --> wish
trouble --> certain
trouble --> treat
trouble --> outside
trouble --> click
trouble --> venue
trouble --> intend
trouble --> language
trouble --> track
trouble --> legitimate
trouble --> acquire
trouble --> close
trouble --> least
trouble --> phone
trouble --> leave
trouble --> practice
trouble --> log
trouble --> allow
trouble --> text
trouble --> grant
trouble --> select
trouble --> current
trouble --> play
trouble --> entity
trouble --> unlawful
trouble --> store
trouble --> image
trouble --> interactive
trouble --> year
trouble --> even
chargeback --> policy
chargeback --> cannot
chargeback --> amendment
chargeback --> regulation
chargeback --> perform
chargeback --> conduct
chargeback --> product
chargeback --> anything
chargeback --> due
chargeback --> exceed

doing for  DuckDuckGo.txt
doing for  Facebook.txt
doing for  Flickr.txt
doing for  Github.txt
doing for  Jabong.txt
doing for  Loco.txt
doing for  Microsoft.txt
doing for  Paytm.txt
doing for  Youtube.txt
Accuracy for  DubsmashSummary.txt  and  DubsmashGroundTruth.txt  is :  70.37037037037037 %
Accuracy for  DuckDuckGoSummary.txt  and  DuckDuckGoGroundTruth.txt  is :  60.0 %
Accuracy for  FacebookSummary.txt  and  FacebookGroundTruth.txt  is :  88.23529411764706 %
Accuracy for  FlickrSummary.txt  and  FlickrGroundTruth.txt  is :  73.68421052631578 %
Accuracy for  GithubSummary.txt  and  GithubGroundTruth.txt  is :  78.125 %
Accuracy for  JabongSummary.txt  and  JabongGroundTruth.txt  is :  77.5 %
Accuracy for  LocoSummary.txt  and  LocoGroundTruth.txt  is :  82.75862068965517 %
Accuracy for  MicrosoftSummary.txt  and  MicrosoftGroundTruth.txt  is :  68.42105263157895 %
Accuracy for  PaytmSummary.txt  and  PaytmGroundTruth.txt  is :  60.86956521739131 %
Accuracy for  YoutubeSummary.txt 